# Task 2 - LFSR

### 1. Define a generator that implements an LFSR. Given a polynomial and an initial state, it generates an infinite stream of bits. 

In [59]:
from functools import reduce
from operator import xor
from itertools import islice

def lfsr_generator(poly, state=None):

    if state is None: 
        s = [True for i in range(max(poly)+1)]
    else: 
        s = [i == '1' for i in state] #Gir True for i==1 ikkesant

    p = [False for i in range(max(poly)+1)]
    for i in range(len(p)):
        if i in poly:
            p[i] = True
    p.reverse()
    p.pop()

    while True: 

        b = s[0]
        anded = []
        for i in range(len(p)):
            anded.append(s[i] and p[i])
        fb = reduce(xor, anded)

        s.pop(0)
        s.append(fb)
        
        yield b

    #''.join([f'{bit:d}' for bit in b])

lfsr = lfsr_generator([3,1,0], state="111")
for b in islice(lfsr, 8):
    print(f'{b:d}', end='')

11101001

### 1. ALternative solution

In [1]:
def lfsr_generator(poly, state=None, verbose=False):

    length = max(poly)  #LFSR length as the max degree of the polynomial
    poly = [i in poly for i in range(length+1)]     #turn poly into a list of booleans

    if state is None:   #default value for state is all ones (True)
        state = [True for _ in range(length)]
    else:   #convert integer into list of bool
        state = [bool(int(s)) for s in (f'{state:0{length}}')[::-1]]

    # === initial state ===
    # compute output bit and feedback bit
    output = state[0]
    feedback = reduce(xor, compress(state[::-1], poly[1:]))
    if verbose:     #print initial state
        print('state    b   fb')
        print_lfsr(state, output, feedback)

    # === infinite loop ===
    while True: 
        state = state[1:] + [feedback]
        output = state[0]
        feedback = reduce(xor, compress(state[::-1], poly[1:]))
        if verbose: #print current state
            print_lfsr(state, output, feedback)
        
        yield output #return current output

poly = [3, 1, 0]
state = 0x7
lfsr = lfsr_generator(poly, state, verbose=True)


### 2. Transform the LFSR generator in an iterator, so that it is possible to access to the internal state as an attribute of the class.

In [25]:
class LFSR(object):
    ''' class docstring '''

    def __init__(self, poly, state=None):
        ''' constructor docstring '''

        self.length = len(poly) 
        self.output = None
        self.feedback = None

        #self.poly = self.p
        self.p = [False for i in range(max(poly)+1)]
        for i in range(len(self.p)):
            if i in poly:
                self.p[i] = True
        self.p.reverse()
        self.p.pop()

        #self.state = self.s
        if state is None: 
            self.s = [True for i in range(max(poly)+1)]
        else: 
            s = [i == '1' for i in state]
    
    def __iter__(self): 
        return self

    def __next__(self): 
        anded = []
        for i in range(len(self.p)):
            anded.append(self.s[i] and self.p[i])
        fb = reduce(xor, anded)
        self.output = self.s[0]
        self.s.pop(0)
        self.s.append(fb)
        return self.output

    def run_steps(self, N=1): 
        list_of_bool = []
        for i in range(N):
            list_of_bool.append(self.__next__())
        return list_of_bool 
    
    def cycle(self, state=None): 
        N = 2^self.length-1
        list_of_bool = self.run_steps(N)
        return list_of_bool

    def __str__(self):
        return ''

lfsr = LFSR([3, 1, 0])
print(lfsr.cycle())


[]


### 3. Define a function that implements the Berlekamp-Massey algorithm which finds the shortest LFSR that can generate the input bit stream.

In [10]:
from functools import reduce
from operator import xor
from itertools import islice, compress

def berlekamp_massey(b): #DENNE ER IKKE RIKTIG!!!!!!

    b = str(b)
    N = len(b)
    P = [1] #polynomial, output, list of bools
    Q = 1 #polynomial used internally, list of bools
    m = 0   #interger, degree of the polynomial P(x) 
    r = 1   #integer, how many bits so far, "round number"

    for t in range(N-1):
        print('=== Ny runde===')
        print('t: ', t)
        anded = []
        """
        x_r = [0] * N
        """
        for j in range(m+3): 
            print('j: ',j)
            anded.append(P[j] and int(b[t-j]))
            print('anded: ', anded)
            d = reduce(xor, anded)
            print('d: ', d)
            if d == 1:
                print('1st if condition')
                if 2*m <= t: 
                    print('2nd if condition')
                    R = P #save old P as R
                    print('Old P:', P)
                    P.insert(0, Q) #add next ledd Q to final polynomial P 
                    print('New P: ', P)
                    """
                    x_r[N-r-1] = 1
                    Qx_r = bin_to_int(Q) * bin_to_int(x_r)
                    P_int = bin_to_int(P)
                    P_int = np.bitwise_xor(P_int, Qx_r)
                    P = int_to_bin(P_int, N)
                    """
                    Q = R   #save Q as new old P
                    m = t + 1 - m   
                    r = 0

                else:
                    P.insert(0, Q)
                    print('New P: ', P)
                    """
                    x_r[N-r-1] = 1
                    Qx_r = bin_to_int(Q) * bin_to_int(x_r)
                    P_int = bin_to_int(P)
                    P_int = np.bitwise_xor(P_int, Qx_r)

                    P = int_to_bin(P_int, N)
                    """
            else: 
                P.insert(0, 0)
            r += 1
        return P 
        """ 
        return P[::-1][:m+1]
        """

print(berlekamp_massey(10100111))

=== Ny runde===
t:  0
j:  0
anded:  [1]
d:  1
1st if condition
2nd if condition
Old P: [1]
New P:  [1, 1]
j:  1
anded:  [1, 1]
d:  0
j:  2
anded:  [1, 1, 1]
d:  1
1st if condition
New P:  [[...], 0, 1, 1]
[[...], 0, 1, 1]


In [ ]:
def print_poly(polynomial): 
    if polynomial[0] == 1:
        result = '1 + '
    else: 
        result = ''
    for idx,i in enumerate(polynomial[1:]):
        if i == 1: 
            result += 'x^{}'.format(idx+1)
            if idx != (len(polynomial)-2):
                result += ' + '
    return result

In [4]:
from functools import reduce
from operator import xor
from itertools import islice, compress
import numpy as np

def berlekamp_massey(b): 

    N = len(b)
    P = [0] * N 
    P[N-1] = 1
    m = 0
    Q = [0] * N
    Q[N-1] = 1
    r = 1

    for t in range(N): 
        anded = []
        x_r = [0] * N
        for j in range(m+1):
            P_inv = P[::-1]
            anded.append(P_inv[j] and b[t-j])
            d = reduce(xor, np.array(anded))
        if d == 1: 
            if 2*m <= t: 
                R = P
                x_r[N-r-1] = 1
                Qx_r = bin_to_int(Q) * bin_to_int(x_r)
                P_int = bin_to_int(P)
                P_int = np.bitwise_xor(P_int, Qx_r)

                P = int_to_bin(P_int, N)
                Q = R
                m = t + 1 - m
                r = 0
            else: 
                x_r[N-r-1] = 1
                Qx_r = bin_to_int(Q) * bin_to_int(x_r)
                P_int = bin_to_int(P)
                P_int = np.bitwise_xor(P_int, Qx_r)

                P = int_to_bin(P_int, N)
        r = r+1
    return P #[::-1][:m+1]

print(berlekamp_massey('10100111'))

[0, 0, 0, 0, 0, 0, 0, 1]


### 4. Transform the function implementing the Berlekamp-Massey algorithm into a class that can be applied in a streamingway.

In [ ]:
class BerlekampMassey():

    P(x) = 
    Q(x) = 
    m = 
    r = 
    
    def __init__(self):
    # do stuff
    self.poly = ... 
    
    def __call__(self, bit):
    # do stuff
    return self.poly

### 4. Alternative solution

In [ ]:
from functools import reduce
from operator import xor
from itertools import islice, compress
import numpy as np

class BerlekampMassey():
    
    def __init__(self):

        self.P = [1]
        self.m = 0
        self.Q = [1]
        self.r = 1
        self.t = 0 
        self.b = []
        self.poly = 0
    
    def __call__(self, bit):
        
        self.append(b)
        anded = []
        for j in range(self.m+1):
            P_inv = self.P[::-1]
            anded.append(P_inv[j] and self.b[self.t-j])
            d = reduce(xor, np.array(anded))
        if d == 1: 
            if 2*self.m <= self.t: 
                self.R = self.P
                x_r = [0] * (self.r+1)
                x_r[len(x_r)-self.r-1] = 1
                Qx_r = bin_to_int(self.Q) * bin_to_int(x_r)
                P_int = bin_to_int(self.P)
                P_int = np.bitwise_xor(P_int, Qx_r)

                self.P = int_to_bin(P_int, len(self.b)+1)
                self.Q = self.R
                self.m = self.t + 1 - self.m
                self.r = 0
            else: 
                x_r = [0] * self.r
                x_r[len(x_r)-self.r-1] = 1
                Qx_r = bin_to_int(self.Q) * bin_to_int(x_r)
                P_int = bin_to_int(self.P)
                P_int = np.bitwise_xor(P_int, Qx_r)

                P = int_to_bin(P_int, N)
        
        # vet ikke hva mer som skal stå her

        return self.poly